## PointNet to Classify Proteins

### Dataset study

In [2]:
# Getting the data
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys

home_path = '/home/giorgio/venvs/SHREC'
if home_path not in sys.path:
    sys.path.append(home_path)

from pandas import DataFrame
import pandas as pd

from pathlib import Path

from datasetStudy import *

debug_pointnet = False

# Multiple function here depends on thi variable
#root = '/mnt/dataset/shrec-2025-protein-classification/v2-20250331' 
base_path = Path('/mnt/')
#base_path = Path('/mnt/raid1')
root = base_path / 'dataset/shrec-2025-protein-classification/v2-20250331'

#train_data = os.listdir(os.path.join(root, 'train'))
#train_data_cls = pd.read_csv('datasets/train_set-all.csv', sep=',', index_col=0)

train_data = os.listdir(root / 'train')
train_data_cls = pd.read_csv('../datasets/train_set-all.csv', sep=',', index_col=0)

print(train_data[:5])
print()
print(train_data_cls.info())
train_data_cls

['8ugd_8:R:3U_model1.vtk', '8h0v_18:R:c_model1.vtk', '3j3q_1:DX:4F_model1.vtk', '4u4u_23:XC:d1_model1.vtk', '6rny_4:H:H_model1.vtk']

<class 'pandas.core.frame.DataFrame'>
Index: 9244 entries, 0 to 9243
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   protein_id        9244 non-null   object
 1   class_id          9244 non-null   int64 
 2   number_of_points  9244 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 288.9+ KB
None


,protein_id,class_id,number_of_points
0,8ugd_8:R:3U_model1,96,5916
1,8h0v_18:R:c_model1,86,10078
2,3j3q_1:DX:4F_model1,8,18432
3,4u4u_23:XC:d1_model1,83,8242
4,6rny_4:H:H_model1,34,9204
...,...,...,...
9239,3j3y_1:HL:6R_model1,8,18342
9240,4u4y_15:O:C3_model1,91,12976
9241,7w31_26:AA:c_model1,18,16454
9242,3j4k_1:E:E_model1,90,23188


In [3]:
src = root / 'train-xz'
mountpoint = '/tmp/ramdrive'
size_in_gb = 3

def create_ramdrive(src, mountpoint, size_in_gb):
    if Path(mountpoint).exists() or Path(mountpoint).is_dir():
        print(f'Ramdrive already created, exiting...')
        return
    
    !sudo mkdir -p "{mountpoint}"
    !sudo mount -o size="{size_in_gb}"G -t tmpfs none "{mountpoint}"
    !rsync -a --progress "{src}" "{mountpoint}"
    
def unmount_ramdrive(mountpoint):
    !sudo umount "{mountpoint}"
    !sudo rmdir "{mountpoint}"

In [4]:
create_ramdrive(src, mountpoint, size_in_gb)
#unmount_ramdrive(mountpoint)

Ramdrive already created, exiting...


In [5]:
possible_disconnected_mesh(train_data_cls, 96, False)

disconnected_dict = {}
for idx in range(97):
    disconnected_dict[idx] = possible_disconnected_mesh(train_data_cls, idx)

#print(disconnected_dict)

_, damaged = possible_disconnected_mesh(train_data_cls, 8, True); damaged

,protein_id,class_id,number_of_points


In [6]:
dist = cls_distribution(train_data_cls)
inspect_distribution(dist, l_lim=0, u_lim=5)

Classes which have between 0 and 5 element: 16/97


In [7]:
dist_all = cls_distribution(train_data_cls)

# Filter the number of points
raw_train_dataframe_f1 = number_of_point_filter(train_data_cls, 1000)
dist_f1 = cls_distribution(raw_train_dataframe_f1)


# Filter according to the number of samples in classes
raw_train_dataframe_f2 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=10)
dist_f2 = cls_distribution(raw_train_dataframe_f2)

raw_train_dataframe_f3 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=10, u_cut_off=100)
dist_f3 = cls_distribution(raw_train_dataframe_f3)

raw_train_dataframe_f4 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=10, u_cut_off=50)
dist_f4 = cls_distribution(raw_train_dataframe_f4)

raw_train_dataframe_f5 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=100)
dist_f5 = cls_distribution(raw_train_dataframe_f5)

raw_train_dataframe_f6 = number_of_class_filter(raw_train_dataframe_f1, l_cut_off=0, u_cut_off=450)
dist_f6 = cls_distribution(raw_train_dataframe_f6)

distributions = [dist_all, dist_f1, dist_f2, dist_f3, dist_f4, dist_f5, dist_f6]
for idx in range(len(dist_all)):
    output = f'cls {idx}: '
    for dist in distributions:
        output += f'{print_dist(dist, idx)}\t'
    print(output)

print(f'{len(train_data_cls) = }')
print(f'{len(raw_train_dataframe_f1) = }')
print(f'{len(raw_train_dataframe_f2) = }')
print(f'{len(raw_train_dataframe_f3) = }')
print(f'{len(raw_train_dataframe_f4) = }')
print(f'{len(raw_train_dataframe_f5) = }')

cls 0: 101	101	101	0	0	101	101	
cls 1: 3	3	0	0	0	0	3	
cls 2: 17	17	17	17	17	0	17	
cls 3: 21	21	21	21	21	0	21	
cls 4: 15	15	15	15	15	0	15	
cls 5: 127	127	127	0	0	127	127	
cls 6: 25	25	25	25	25	0	25	
cls 7: 75	75	75	75	0	0	75	
cls 8: 2054	2054	2054	0	0	2054	0	
cls 9: 66	66	66	66	0	0	66	
cls 10: 14	14	14	14	14	0	14	
cls 11: 43	43	43	43	43	0	43	
cls 12: 10	10	10	10	10	0	10	
cls 13: 22	22	22	22	22	0	22	
cls 14: 489	489	489	0	0	489	0	
cls 15: 89	89	89	89	0	0	89	
cls 16: 154	154	154	0	0	154	154	
cls 17: 116	116	116	0	0	116	116	
cls 18: 76	76	76	76	0	0	76	
cls 19: 42	42	42	42	42	0	42	
cls 20: 7	7	0	0	0	0	7	
cls 21: 74	74	74	74	0	0	74	
cls 22: 58	58	58	58	0	0	58	
cls 23: 10	10	10	10	10	0	10	
cls 24: 10	10	10	10	10	0	10	
cls 25: 143	143	143	0	0	143	143	
cls 26: 2	2	0	0	0	0	2	
cls 27: 3	3	0	0	0	0	3	
cls 28: 18	18	18	18	18	0	18	
cls 29: 14	14	14	14	14	0	14	
cls 30: 3	3	0	0	0	0	3	
cls 31: 33	33	33	33	33	0	33	
cls 32: 93	93	93	93	0	0	93	
cls 33: 126	126	126	0	0	126	126	
cls 34: 73	73	73	73	0	0	73	
c

In [8]:
dist_f6_available_classes = []
for key,value in dist_f6.items():
    if value:
        dist_f6_available_classes.append(key)
print(len(dist_f6_available_classes), dist_f6_available_classes)

93 [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96]


In [9]:
symmetria_flist = list((root / 'train-symmetria-xz').glob('*.xz'))
symmetria_flist = [(fpath.name[3:-3], fpath.name[0:2], None) for fpath in symmetria_flist]

In [10]:
symmetria_df = pd.DataFrame(symmetria_flist, columns=['protein_id', 'class_id', 'number_of_points'])

In [11]:
symmetria_df

,protein_id,class_id,number_of_points
0,000002-astroid-clean,00,None
1,000011-astroid-gaussian,00,None
2,000012-astroid-gaussian,00,None
3,000021-astroid-undersampling+gaussian,00,None
4,000032-astroid-gaussian,00,None
...,...,...,...
20100,099907-citrus-undersampling+gaussian,01,None
20101,099922-citrus-gaussian,01,None
20102,099957-citrus-undersampling,01,None
20103,099976-citrus-gaussian,01,None


In [12]:
cls_selected = [8, 90, 56, 14, 61, 70, 86, 5, 16, 25, 54, 62, 74, 83, 91, 92, 17, 45, 87, 71]

df_3 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:2],  number_of_proteins=500)
df_4 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:10], number_of_proteins=100)
df_5 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:20], number_of_proteins=50)
df_6 = create_dataframe(raw_train_dataframe_f1, class_ids=cls_selected[:20], number_of_proteins=100)
df_7 = create_dataframe(raw_train_dataframe_f6, class_ids=dist_f6_available_classes, number_of_proteins=100000)
df_8 = create_dataframe(raw_train_dataframe_f1, class_ids=[40, 88], number_of_proteins=10000)

'''
df_3.to_csv('datasets/train_set-2_cls-1000_images.csv')
df_4.to_csv('datasets/train_set-10_cls-1000_images.csv')
df_5.to_csv('datasets/train_set-20_cls-1000_images.csv')
df_6.to_csv('datasets/train_set-20_cls-2000_images.csv')
df_7.to_csv('datasets/train_set-all-cls_except-cls-larger-than-450.csv')
symmetria_df.to_csv('datasets/train_set-symmetria-100k-easy-astroid-citrus-10k-samples.csv')
df_8.to_csv('datasets/train_set-2_cls_40_88-192_proteins.csv')
'''



"\ndf_3.to_csv('datasets/train_set-2_cls-1000_images.csv')\ndf_4.to_csv('datasets/train_set-10_cls-1000_images.csv')\ndf_5.to_csv('datasets/train_set-20_cls-1000_images.csv')\ndf_6.to_csv('datasets/train_set-20_cls-2000_images.csv')\ndf_7.to_csv('datasets/train_set-all-cls_except-cls-larger-than-450.csv')\nsymmetria_df.to_csv('datasets/train_set-symmetria-100k-easy-astroid-citrus-10k-samples.csv')\ndf_8.to_csv('datasets/train_set-2_cls_40_88-192_proteins.csv')\n"

In [13]:
tmp_df = pd.read_csv('/home/giorgio/venvs/SHREC/datasets/train_set-2_cls-194_images.csv', index_col=0)
len(df_8)

192

In [14]:
df = raw_train_dataframe_f1[raw_train_dataframe_f1['class_id'] == 10]
print(len(df))
visualize_mesh(raw_train_dataframe_f1, cls=10, idx=3)

14
PolyData (0x7b6e1d168940)
  N Cells:    115068
  N Points:   57516
  N Strips:   0
  X Bounds:   -5.945e+01, 5.574e+01
  Y Bounds:   -4.762e+01, 5.398e+01
  Z Bounds:   -3.284e+01, 3.595e+01
  N Arrays:   3


Widget(value='<iframe src="http://localhost:44139/index.html?ui=P_0x7b6e86bf6540_0&reconnect=auto" class="pyvi…

### Augmentations

In [15]:
from format import Text
import torch
from symmetria.transformations import *
from symmetria.shapes import BenchmarkShape

In [16]:
points_rectangle = np.array([[0, 2, 0, 0, 2, 2, 0, 2],
                             [0, 0, 1, 0, 1, 0, 1, 1],
                             [0, 0, 0, 1, 0, 1, 1, 1]]).astype(np.float32)

points_rectangle_t = torch.from_numpy(points_rectangle)

rot = random_rotation_matrix(rotate_x=False, rotate_z=False)

shapebench = BenchmarkShape(points_rectangle_t)
#shapebench.apply_rotation(torch.from_numpy(rot))
#shapebench.apply_traslation(1,1,0)
#shapebench.apply_uniform_noise(100, 1)
#shapebench.apply_gaussian_noise(100, 1)

rect = pv.PolyData(shapebench.points.numpy().T)
#rect.plot()

In [17]:
from random import random

class RotateAroundZero():
    def __init__(self, p=0.5, rot=None):
        self.p = p
        self.rot = rot

    def __call__(self, points):
        if random() < self.p:
            if self.rot == None:
                rot = torch.from_numpy(random_rotation_matrix())
            else:
                rot = self.rot
        
            rot = rot.to(torch.device(points.device))

            ones = torch.ones((1,points.shape[0]), device=points.device)
            coords = torch.concatenate((torch.transpose(points, 0, 1), ones))
            
            coords = rot@coords
            
            points = torch.transpose(coords[:3,:], 0, 1)

        return points
    
    def __repr__(self):
        return f'RotateAroundZero(p={repr(self.p)}, rot={repr(self.rot)})'
    
class Translate():
    def __init__(self, p=0.5, shift=None, scale=1):
        self.p = p
        self.shift = shift
        self.scale = scale

    def __call__(self, points):        
        if random() < self.p:
            if self.shift == None:
                shift = (torch.rand(1, device=points.device) * self.scale,
                         torch.rand(1, device=points.device) * self.scale,
                         torch.rand(1, device=points.device) * self.scale)
            else:
                shift = self.shift
            
            for i in range(len(shift)):
                points[:,i] += shift[i]
                   
        return points
    
    def __repr__(self):
        return f'Translate(p={repr(self.p)}, shift={repr(self.shift)}, scale={repr(self.scale)})'
    
class UniformNoise():
    def __init__(self, n, T, p=0.5):
        self.p = p
        self.n, self.T = n, T
    
    def __call__(self, points):
        if random() < self.p:
            num_points = points.shape[0]

            indices = torch.randperm(num_points, device=points.device)[:self.T]

            points[indices,:] = points[indices,:] + (2*torch.rand(self.T, 3, device=points.device)-1)/self.n
        
        return points
    
    def __repr__(self):
        return f'UniformNoise(n={repr(self.n)}, T={repr(self.T)}, p={repr(self.p)})'
    
class GaussianNoise():
    def __init__(self, n, T, p=0.5):
        self.p = p
        self.n, self.T = n, T
    
    def __call__(self, points):
        if random() < self.p:
            num_points = points.shape[0]

            indices = torch.randperm(num_points, device=points.device)[:self.T]
    
            points[indices,:] = points[indices,:] + torch.rand(self.T, 3, device=points.device)/self.n

        return points
    
    def __repr__(self):
        return f'GaussianNoise(n={repr(self.n)}, T={repr(self.T)}, p={repr(self.p)})'

In [18]:
points_rectangle = np.array([[0, 2, 0, 0, 2, 2, 0, 2],
                             [0, 0, 1, 0, 1, 0, 1, 1],
                             [0, 0, 0, 1, 0, 1, 1, 1]]).astype(np.float32).T

points_rectangle_t = torch.from_numpy(points_rectangle).to(torch.device('cuda:0'))

translate = Translate(p=1)
print(translate)
points_rectangle_t = translate(points_rectangle_t)
print(translate)
rect = pv.PolyData(points_rectangle)
#rect.plot()

rect_tfm = pv.PolyData(points_rectangle_t.cpu().numpy())
#rect_tfm.plot()

Translate(p=1, shift=None, scale=1)
Translate(p=1, shift=None, scale=1)


### DataSet and DataLoader

#### Dataset

In [19]:
import numpy as np
import lzma
from torch.utils.data import Dataset, DataLoader
from symmetria.transforms.RandomSampler import RandomSampler
from symmetria.transforms.UnitSphereNormalization import UnitSphereNormalization
from tqdm import tqdm

class ProteinDataset(Dataset):
    def __init__(self, data_df, tfms, root=root, extention='xz', train=True):
        super().__init__()
        self.df = data_df
        self.tfms = tfms
        self.extention = extention
        
        self.train = train
        if self.train:
            if self.extention == 'vtk':
                self.root = os.path.join(root, 'train')

            elif self.extention == 'xz':
                self.root = os.path.join(root, 'train-xz') # TODO roll back
                #self.root = os.path.join(root, 'train-xz')
            else:
                raise UserWarning('Extention not supported')
            
        else:
            raise UserWarning('Not yet implemented for the test dataset')
        
        self.encode_label()
        self.loader()
    
    def loader(self):
        self.data = []
        
        for index in tqdm(self.df.index):
            protein, cls, nop = self.df['protein_id'].loc[index], self.df['class_id'].loc[index], self.df['number_of_points'].loc[index]

            cls_t = torch.tensor(self.encoded_cls[cls]).to(torch.device('cuda:0'))

            if self.extention == 'vtk':
                point_cloud = self.get_vtk_points(protein)

            elif self.extention == 'xz':
                point_cloud = self.get_xz_points(protein, cls, nop)

            point_cloud_t = torch.from_numpy(point_cloud).to(torch.device('cuda:0'))     

            point_cloud_t, cls_t = point_cloud_t.type(torch.float32), cls_t.type(torch.float32)
            self.data.append((point_cloud_t, cls_t))

    def get_vtk_points(self, name):
        prot_file = name + '.' + self.extention
        prot_file = os.path.join(self.root, prot_file)
        
        prot_mesh = pv.read(prot_file)

        return prot_mesh.points

    def get_xz_points(self, name, cls, nop=None, debug=False):
        cls = str(cls)
        nop = str(nop) if nop and nop != 'nan' else None

        while len(cls) < 2:
            cls = '0' + cls
        
        if nop and nop != 'nan':
            while len(nop) < 6:
                nop = '0' + nop

        if nop and nop != 'nan':
            prot_file = cls + '-' + nop +  '-' + name.replace(':', '+') + '.' + self.extention
        else:
            prot_file = cls + '-' + name.replace(':', '+') + '.' + self.extention
        if debug:
            print(f'ProteinDataset is loading files from {self.root}')
        prot_file = os.path.join(self.root, prot_file)        
        
        with lzma.open(prot_file, 'rt') as f:
            point_cloud = np.loadtxt(f)

        return point_cloud

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index, do_transform=True):
        prot, cls = self.data[index]
        
        if do_transform:
            for tfm in self.tfms:
                prot = tfm(prot)

        # NOTE: here we transpose the points (from Nx3 to 3xN) to then pass them to the PoinNet encoder
        prot = torch.transpose(prot, 0, 1)
        return (prot, cls)
    
    def encode_label(self):
        self.encoded_cls = {}
        
        prot_clss = np.sort(self.df['class_id'].unique())
        
        #One hot encoding
        '''
        for idx, cls in enumerate(prot_clss):
            self.encoded_cls[cls] = np.eye(len(prot_clss))[idx]
        '''

        for idx, cls in enumerate(prot_clss):
            self.encoded_cls[cls] = int(idx)

    def render_pointcloud(self, index):
        prot, _ = self.data[index]
        prot = torch.transpose(prot, 0, 1).numpy()
        cloud = pv.PolyData(prot)
        print(cloud)
        cloud.plot()
    

transforms = [UnitSphereNormalization(),
              Translate(p=0.8, scale=1),
              RotateAroundZero(p=0.8),
              GaussianNoise(n=100, T=2500, p=0.8),
              RandomSampler(sample_size=5000)]


def generate_train_valid_set(df, tfms, val_pct, root=root, seed=42, **kwargs):
    '''
    Can also take some other arguments to be passed to the dataset initializer

            -> path (str): path to the parent directory containing the train files 
    '''

    train_size = int(len(df) * (1 - val_pct))    
    df_train = df.sample(train_size, random_state=seed)
    df_valid = df.drop(df_train.index)

    return ProteinDataset(df_train, tfms, root=root, **kwargs), ProteinDataset(df_valid, tfms, root=root, **kwargs)

'''
def generate_train_valid_set_boring(df, val_pct, seed=42):
    train_size = int(len(df) * val_pct)    
    df_train = df.sample(train_size, random_state=seed)
    df_valid = df.drop(df_train.index)

    return df_train, df_valid
'''

'\ndef generate_train_valid_set_boring(df, val_pct, seed=42):\n    train_size = int(len(df) * val_pct)    \n    df_train = df.sample(train_size, random_state=seed)\n    df_valid = df.drop(df_train.index)\n\n    return df_train, df_valid\n'

In [20]:
example_dataframe = create_dataframe(raw_train_dataframe_f1, class_ids=[2, 30, 10], number_of_proteins=100000)
example_set_train, example_set_valid = generate_train_valid_set(example_dataframe, transforms, 0.2)

100%|██████████| 7/7 [00:00<00:00, 62.09it/s]


In [21]:
example_set_train.__getitem__(0, do_transform=False)

(tensor([[  5.6880,   9.1380,   5.1850,  ...,  18.7270,  19.7440,  19.8410],
         [-20.2360, -20.3460, -20.1250,  ...,  -9.1310, -11.6640, -10.8590],
         [-13.3830, -13.4790, -12.5080,  ...,  23.6170,  23.4130,  23.4920]],
        device='cuda:0'),
 tensor(1., device='cuda:0'))

In [22]:
prot = example_set_train.__getitem__(0, do_transform=False)

In [23]:
print(f'{Text(prot, 'prot'):content}')

prot type = <class 'tuple'>
prot len = 2
prot = (tensor([[  5.6880,   9.1380,   5.1850,  ...,  18.7270,  19.7440,  19.8410],
        [-20.2360, -20.3460, -20.1250,  ...,  -9.1310, -11.6640, -10.8590],
        [-13.3830, -13.4790, -12.5080,  ...,  23.6170,  23.4130,  23.4920]],
       device='cuda:0'), tensor(1., device='cuda:0'))



In [24]:
print(f'{Text(prot[0], 'prot[0]'):content}')

prot[0] type = <class 'torch.Tensor'>
prot[0] device = cuda:0
prot[0] dtype = torch.float32
prot[0] shape = torch.Size([3, 23402])
prot[0] = tensor([[  5.6880,   9.1380,   5.1850,  ...,  18.7270,  19.7440,  19.8410],
        [-20.2360, -20.3460, -20.1250,  ...,  -9.1310, -11.6640, -10.8590],
        [-13.3830, -13.4790, -12.5080,  ...,  23.6170,  23.4130,  23.4920]],
       device='cuda:0')



In [25]:
print(f'{prot[0].min() = } - {prot[0].max() = }')

prot[0].min() = tensor(-39.9160, device='cuda:0') - prot[0].max() = tensor(38.2570, device='cuda:0')


In [26]:
unit_normalizer = UnitSphereNormalization()
prot, _ = example_set_train.__getitem__(0, do_transform=False)
prot = torch.transpose(prot, 0, 1)
norm_prot = unit_normalizer(prot)

In [27]:
print(f'{Text(norm_prot, 'norm_prot'):content}')

norm_prot type = <class 'torch.Tensor'>
norm_prot device = cuda:0
norm_prot dtype = torch.float32
norm_prot shape = torch.Size([23402, 3])
norm_prot = tensor([[ 0.1330, -0.4869, -0.3020],
        [ 0.2093, -0.4893, -0.3042],
        [ 0.1219, -0.4844, -0.2827],
        ...,
        [ 0.4213, -0.2413,  0.5160],
        [ 0.4438, -0.2973,  0.5115],
        [ 0.4460, -0.2795,  0.5132]], device='cuda:0')



In [28]:
print(f'{norm_prot.min() = } - {norm_prot.max() = }')

norm_prot.min() = tensor(-0.9220, device='cuda:0') - norm_prot.max() = tensor(0.8064, device='cuda:0')


In [29]:
print(f'{unit_normalizer.centroid.min() = } - {unit_normalizer.centroid.max() = }')

unit_normalizer.centroid.min() = tensor(-0.3297, device='cuda:0') - unit_normalizer.centroid.max() = tensor(1.7850, device='cuda:0')


In [30]:
unit_normalizer.centroid.shape

torch.Size([3])

In [31]:
points = prot[0]

In [32]:
points

tensor([  5.6880, -20.2360, -13.3830], device='cuda:0')

### PointNet class

In [60]:
import importlib
import symmetria
importlib.reload(symmetria)
importlib.reload(symmetria.decoders)
importlib.reload(symmetria.decoders.prediction_head)

AttributeError: module 'symmetria' has no attribute 'decoders'

#### PointNet

In [33]:
import torch.nn as nn
import torch.nn.functional as F
from symmetria.encoders.pointnet_encoder import PointNetEncoder

from symmetria.encoders.pointnext.pointnext_encoder_parameters import *
from symmetria.encoders.pointnext.pointnext_encoder import PointNeXt

from symmetria.decoders.prediction_head import PredictionHead

if debug_pointnet:
    bs, sz = 1, 2048
    encoder = PointNetEncoder(use_bn=False)
    mock_x = torch.randn(bs, 3, sz)
    output = encoder.forward(mock_x)
    print(f'{Text(output, 'output'):inspect}')

    decoder = PredictionHead(1024, 96)
    output_decoder = decoder.forward(output)
    print(f'{Text(output_decoder, 'output_decoder'):inspect}')

class PointNet(nn.Module):
    def __init__(self, output_size, max_points, use_bn=False, encoder='pointnet'): # make it prettier like in segmenter
        super().__init__()
        
        if encoder in POINTNEXT_MODEL_CONFIG:
            model_cfg = POINTNEXT_MODEL_CONFIG[encoder]     # 'PointNeXt_B' (21.5 M), 'PointNeXt_L2' (32.0 M), 'PointNeXt_XXL' (73.8 M)
            self.encoder = PointNeXt(model_cfg)
            self.encoder_output_size = output_size          # because of the adapter head (e.g. 2048 -> 1024) within the encoder
            print(f"Using PointNeXt {model_cfg} - encoder: {self.encoder}")
        else:
            self.encoder = PointNetEncoder(use_bn)
            print(f"Using PointNet as encoder - encoder: {self.encoder}")
        
        self.max_points = max_points

        self.input_size = self.get_input_size()        
        self.output_size = output_size

        print(f'Creating a PredictionHead with {self.input_size} as input and {self.output_size} as output')
        self.decoder = PredictionHead(self.input_size, self.output_size, use_bn, use_relu=True)
        
        #self.softmax = nn.Softmax(dim=1)

    def get_input_size(self):
        mock_x = torch.randn(1, 3, self.max_points)
        return self.encoder(mock_x).shape[-1]

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        #x = self.softmax(x)

        return x #SoftMax already inside the CrossEntropyLoss

### Config Definition

In [35]:
from datetime import datetime
from torch import optim

time_now = datetime.now()
time_now = time_now.strftime('%d%m%Y_%H%M%S')
_folder = !pwd

env = {}

### Debug
env['debug_loss'] = True

env['project_folder'] = _folder[0]
env['project'] = 'shrec-2025'
env['run'] = env['project'] + '_' + time_now
#env['output_dir'] = os.path.join(env['project_folder'], 'wandb', 'run_' + env['run'])
if 'output' not in os.listdir(env['project_folder']):
    os.mkdir('output')
env['output_dir'] = Path(env['project_folder']) / 'output'

env['max_points'] = 10000

#env['dataset'] = 'train_set-2_cls-1000_images.csv' 
#env['dataset'] = 'train_set-all.csv'
#env['dataset'] = 'train_set-all-cls_except-cls-larger-than-450.csv'
#env['dataset'] = 'train_set-symmetria-100k-easy-astroid-citrus-10k-samples.csv'
#env['dataset'] = 'train_set-2_cls-1000_images.csv'
env['dataset'] = 'train_set-10_cls-1000_images.csv'
#env['dataset_path'] = '/mnt/dataset/shrec-2025-protein-classification/v2-20250331'
env['dataset_path'] = root

ds_path = Path('../datasets') / env['dataset']
print(f'Creating a test dataset from {ds_path}')

test = pd.read_csv(ds_path, index_col=0)
print(f'Test dataset has {len(test)} rows')

Creating a test dataset from ../datasets/train_set-10_cls-1000_images.csv
Test dataset has 1000 rows


### Model Definition

In [36]:
env['device'] = 'cuda:0'
env['val_pct'] = 0.2
env['augmentations_on'] = True
'''
env['aug'] = [Translate(p=0.8),
              UnitSphereNormalization(),
              RotateAroundZero(p=0.8),
              GaussianNoise(n=10, T=(env['max_points']//2),  p=0.8),
              RandomSampler(sample_size=env['max_points'])]
'''

env['aug'] = [Translate(p=0.5),
              UnitSphereNormalization(),
              RandomSampler(sample_size=env['max_points']),
              GaussianNoise(n=10, T=(env['max_points']//2),  p=0.5),
              ]

#env['aug'] = [UnitSphereNormalization()]
#env['aug'] = []

env['bs'] = 1
env['epochs'] = 100
env['lr'] = 1e-4
env['wd'] = 0.05 # because "PointNeXt is trained with a weight decay of 0.05 for 250 epochs" - nope, too large, doesn't learn!
env['wd'] = 1e-4 # because of this: https://github.com/yanx27/Pointnet_Pointnet2_pytorch/blob/master/train_classification.py

env['encoder']   = 'pointnet'
#env['encoder']  = 'PointNeXt_XXL'
env['model']     = PointNet(output_size=len(test['class_id'].unique()), max_points=env['max_points'], encoder=env['encoder']).to(torch.device(env['device']))

env['model_pth'] = '/tmp/pointnet.pth'
env['pretrain']  = False

if env['pretrain']:
    if not Path(env['model_pth']).exists():
        !wget 'https://github.com/meder411/PointNet-PyTorch/raw/refs/heads/master/classifier_model_state.pth' --output-document "{env['model_pth']}"
    else:
        print(f'{env['model_pth']} already exists, skipping download...')
    torch.load(env['model_pth'], map_location=torch.device(env['device']))

env['loss_func'] = nn.CrossEntropyLoss()
env['optimizer'] = optim.Adam(env['model'].parameters(), lr=env['lr'], weight_decay=env['wd'])

Using PointNet as encoder - encoder: PointNetEncoder(
  (input_transform): TNet(
    (shared_mlps): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (5): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=9, bias=True)
    )
  )
  (feature_transform): TNet(
    (shared_mlps): Sequential(
      (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (5): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
     

In [ ]:
env

{'debug_loss': True,
 'project_folder': '/home/giorgio/venvs/SHREC/notebooks',
 'project': 'shrec-2025',
 'run': 'shrec-2025_03042025_153656',
 'output_dir': PosixPath('/home/giorgio/venvs/SHREC/notebooks/output'),
 'max_points': 10000,
 'dataset': 'train_set-2_cls-1000_images.csv',
 'dataset_path': PosixPath('/mnt/dataset/shrec-2025-protein-classification/v2-20250331'),
 'device': 'cuda:0',
 'val_pct': 0.2,
 'augmentations_on': True,
 'aug': [Translate(p=0.5, shift=None, scale=1),
  GaussianNoise(n=10, T=5000, p=0.5)],
 'bs': 1,
 'epochs': 100,
 'lr': 0.0001,
 'wd': 0.0001,
 'encoder': 'pointnet',
 'model': PointNet(
   (encoder): PointNetEncoder(
     (input_transform): TNet(
       (shared_mlps): Sequential(
         (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
         (1): ReLU()
         (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
         (3): ReLU()
         (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
         (5): ReLU()
       )
       (linear): Sequent

### Training Loop

In [37]:
import wandb

def calculate_loss(logits, target, loss_func):
    output    = torch.argmax(logits, dim=1)
    target    = target.to(torch.int64)
    #softmaxed = torch.softmax(logits, dim=1)
    loss      = loss_func(logits, target)
    return loss, output, target

def train(train_dl, valid_dl, env):
    net = env['model']
    loss_func = env['loss_func']
    optimizer = env['optimizer']

    wandb.init(project=env['project'], name=env['run'], dir=env['output_dir'], config=env)

    for epoch in range(env['epochs']):
        print('/' * 20 + f' Epoch: {epoch + 1} ' + '/' * 20)
        for step, batch in enumerate(tqdm(train_dl)):
            input_p, target = batch

            net.zero_grad()

            logits = net(input_p)
            #print(f'{Text(output, 'output'):content}')
            #print(f'{Text(target, 'target'):content}')
            
            '''
            output = torch.argmax(logits, dim=1)
            target = target.to(torch.int64)
            loss = loss_func(output, target)
            '''
            loss, output, target = calculate_loss(logits, target, loss_func)
            if env['debug_loss'] and step % 1000 == 0:
                print(f'pred: {int(output)} - target: {int(target)} - loss: {loss:.3f} - raw pred: {logits}')
            wandb.log({'epoch': epoch,
                    'loss': loss})
            #print(f'Training loss: {loss}')
            loss.backward()
            optimizer.step()

        print('/' * 20 + ' Validation ' + '/' * 20)
        with torch.no_grad():
            val_loss = []
            acc = []
            for step, batch in enumerate(tqdm(valid_dl)):
                input_p, target = batch
                
                logits = net(input_p)

                '''
                output = torch.argmax(logits, dim=1)
                target = target.to(torch.int64)
                loss = loss_func(output, target).cpu()
                '''
                loss, output, target = calculate_loss(logits, target, loss_func)
                if env['debug_loss'] and step % 1000 == 0:
                    print(f'pred: {int(output)} - target: {int(target)} - loss: {loss:.3f} - raw pred: {logits}')
                
                val_loss.append(loss.cpu())

                #matching = [torch.argmax(i) == torch.argmax(j) for i, j in zip(output, target)]
                matching = [int(i) == int(j) for i, j in zip(output, target)]
                accuracy = matching.count(True)/len(matching)
                
                if env['debug_loss'] and step % 10 == 0:
                    print(f'{matching = } - {accuracy = }')
                acc.append(accuracy)
            wandb.log({'epoch': epoch,
                    'step': step,
                    'val_loss': np.mean(val_loss),
                    'accuracy': np.mean(acc)})

    wandb.finish()

In [38]:
test_train_ds, test_valid_ds = generate_train_valid_set(df=test, tfms=env['aug'], val_pct=env['val_pct'], root=mountpoint)
test_train_dl, test_valid_dl = DataLoader(test_train_ds, batch_size=env['bs'], shuffle=True), DataLoader(test_valid_ds, batch_size=env['bs'], shuffle=True)

100%|██████████| 200/200 [00:01<00:00, 126.73it/s]


In [39]:
train(test_train_dl, test_valid_dl, env)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: g-palmieri4998 (g-palmieri4998-cnr-imati) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


//////////////////// Epoch: 1 ////////////////////


  0%|          | 0/800 [00:00<?, ?it/s]

pred: 5 - target: 1 - loss: 2.298 - raw pred: tensor([[ 5.8811e-04,  1.6078e-02, -5.2709e-05,  3.8626e-02, -4.5133e-03,
          4.5262e-02, -1.5456e-02, -2.9716e-02,  2.0289e-02,  4.3462e-02]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


100%|██████████| 800/800 [00:04<00:00, 160.09it/s]


...

 86%|████████▌ | 171/200 [00:00<00:00, 424.82it/s]

matching = [True] - accuracy = 1.0
matching = [True] - accuracy = 1.0
matching = [False] - accuracy = 0.0
matching = [False] - accuracy = 0.0
matching = [True] - accuracy = 1.0
matching = [True] - accuracy = 1.0
matching = [True] - accuracy = 1.0
matching = [True] - accuracy = 1.0
matching = [True] - accuracy = 1.0


100%|██████████| 200/200 [00:00<00:00, 422.34it/s]

matching = [False] - accuracy = 0.0
matching = [False] - accuracy = 0.0


accuracy,▁▁▂▃▃▄▄▅▄▆▆▅▆▆▆▇▇▆██▇▇▇▇█▇█▇█▇█████▇▇▇█▇
epoch,▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
loss,▄▄▄▁▃▅▄▃▁▂▁▁▃▃▁▁▅▂▂▂▂▁▁▄▁▁▁▁▁▂▁█▂▁▁▁▃▁▁▁
step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅▄▃▂▂▂▂▂▄▄▁▁▂▂▂▂▁▁▂▂▃▆▄▅▅▃▄▅▃▃█▅▇▅▄▅▅▅▆▅
accuracy,0.62
epoch,99
loss,0.01019
step,199
val_loss,2.00443


In [75]:
unmount_ramdrive(mountpoint)